In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import json 
import requests
import timeit
from datetime import datetime

# Let's read our first Dataset energy_annual_generation

In [2]:
energy_annual_generation = pd.read_excel("Resources/annual_generation_state.xls", header=1).iloc[:, : 5]
energy_annual_generation.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,1990,AK,Total Electric Power Industry,Total,5599506.0
1,1990,AK,Total Electric Power Industry,Coal,510573.0
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,974521.0
3,1990,AK,Total Electric Power Industry,Natural Gas,3466261.0
4,1990,AK,Total Electric Power Industry,Petroleum,497116.0


In [3]:
energy_annual_generation['GENERATION (Megawatthours)'] = energy_annual_generation['GENERATION (Megawatthours)'].astype(str)

In [4]:
#let's find our dataset for NY STATE
energy_annual_generation_NY = energy_annual_generation[(energy_annual_generation['STATE'] == 'NY') & \
                         (energy_annual_generation['TYPE OF PRODUCER'] == 'Total Electric Power Industry')].reset_index(drop=True)

energy_annual_generation_NY['YEAR'] = [int(year) for year in energy_annual_generation_NY['YEAR']]
energy_annual_generation_NY['GENERATION (Megawatthours)'] = [int(num.replace('.', '')) for num in energy_annual_generation_NY['GENERATION (Megawatthours)']]

energy_annual_generation_NY.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,1990,NY,Total Electric Power Industry,Total,1353456920
1,1990,NY,Total Electric Power Industry,Coal,259130470
2,1990,NY,Total Electric Power Industry,Hydroelectric Conventional,281882220
3,1990,NY,Total Electric Power Industry,Natural Gas,227243350
4,1990,NY,Total Electric Power Industry,Nuclear,236233560


In [5]:
energy_proportion = []
curr_year, curr_total = 0, 0
for index, row in energy_annual_generation_NY.iterrows():
  if row[0] != curr_year:
    curr_year = row[0]
    curr_total = row[4]
    energy_proportion.append(1)
  else:
    energy_proportion.append(row[4] / curr_total)

energy_annual_generation_NY['Proportion of Energy in Year'] = energy_proportion

# Let's read our second Dataset Energy Sales annual

In [6]:
energy_sales_annual = pd.read_csv("Resources/energy_sales_annual.csv")

energy_sales_annual

,YEAR,Residential,Commercial,Industrial,Transportation,Total
0,2000,426.68,401.08,140.80,362.68,1331.24
1,2001,429.96,404.16,141.89,365.46,1341.47
2,2002,452.70,425.54,149.39,384.80,1412.43
3,2003,456.51,429.12,150.65,388.03,1424.31
4,2004,462.29,434.56,152.56,392.95,1442.36
5,2005,486.35,457.17,160.50,413.40,1517.42
6,2006,483.60,454.58,159.59,411.06,1508.83
7,2007,498.17,468.28,164.40,423.44,1554.29
8,2008,494.02,464.38,163.03,419.92,1541.35
9,2009,488.33,459.03,161.15,415.08,1523.59


In [7]:
# let's merge the 2 tables
NYS_Annual_Energy_Generation_and_Sales = energy_annual_generation_NY[energy_annual_generation_NY['ENERGY SOURCE'] == 'Total'] \
              .merge(right=energy_sales_annual, how='inner', left_on='YEAR', right_on='YEAR')

NYS_Annual_Energy_Generation_and_Sales.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Proportion of Energy in Year,Residential,Commercial,Industrial,Transportation,Total
0,2000,NY,Total Electric Power Industry,Total,1380790750,1.0,426.68,401.08,140.80,362.68,1331.24
1,2001,NY,Total Electric Power Industry,Total,1439145590,1.0,429.96,404.16,141.89,365.46,1341.47
2,2002,NY,Total Electric Power Industry,Total,1395916880,1.0,452.70,425.54,149.39,384.80,1412.43
3,2003,NY,Total Electric Power Industry,Total,1376433160,1.0,456.51,429.12,150.65,388.03,1424.31
4,2004,NY,Total Electric Power Industry,Total,1379647940,1.0,462.29,434.56,152.56,392.95,1442.36


In [8]:
NYS_Annual_Energy_Generation_and_Sales['GENERATION (Megawatthours)'] = (NYS_Annual_Energy_Generation_and_Sales['GENERATION (Megawatthours)'] / 1000000)
NYS_Annual_Energy_Generation_and_Sales.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Proportion of Energy in Year,Residential,Commercial,Industrial,Transportation,Total
0,2000,NY,Total Electric Power Industry,Total,1380.79075,1.0,426.68,401.08,140.80,362.68,1331.24
1,2001,NY,Total Electric Power Industry,Total,1439.14559,1.0,429.96,404.16,141.89,365.46,1341.47
2,2002,NY,Total Electric Power Industry,Total,1395.91688,1.0,452.70,425.54,149.39,384.80,1412.43
3,2003,NY,Total Electric Power Industry,Total,1376.43316,1.0,456.51,429.12,150.65,388.03,1424.31
4,2004,NY,Total Electric Power Industry,Total,1379.64794,1.0,462.29,434.56,152.56,392.95,1442.36


In [9]:
NYS_Annual_Energy_Generation_and_Sales['Energy Surplus'] = NYS_Annual_Energy_Generation_and_Sales['GENERATION (Megawatthours)'] - NYS_Annual_Energy_Generation_and_Sales['Total'] 
NYS_Annual_Energy_Generation_and_Sales.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Proportion of Energy in Year,Residential,Commercial,Industrial,Transportation,Total,Energy Surplus
0,2000,NY,Total Electric Power Industry,Total,1380.79075,1.0,426.68,401.08,140.80,362.68,1331.24,49.55075
1,2001,NY,Total Electric Power Industry,Total,1439.14559,1.0,429.96,404.16,141.89,365.46,1341.47,97.67559
2,2002,NY,Total Electric Power Industry,Total,1395.91688,1.0,452.70,425.54,149.39,384.80,1412.43,-16.51312
3,2003,NY,Total Electric Power Industry,Total,1376.43316,1.0,456.51,429.12,150.65,388.03,1424.31,-47.87684
4,2004,NY,Total Electric Power Industry,Total,1379.64794,1.0,462.29,434.56,152.56,392.95,1442.36,-62.71206


In [13]:
NYS_Annual_Energy_Generation_and_Sales

start_date = 2015
end_date = 2021
mask = (NYS_Annual_Energy_Generation_and_Sales['YEAR'] >= start_date) & (NYS_Annual_Energy_Generation_and_Sales['YEAR'] <= end_date)
NYS_Annual_Energy_Generation_and_Sales= NYS_Annual_Energy_Generation_and_Sales.loc[mask]
NYS_Annual_Energy_Generation_and_Sales

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Proportion of Energy in Year,Residential,Commercial,Industrial,Transportation,Total,Energy Surplus
15,2015,NY,Total Electric Power Industry,Total,1385.81686,1.0,502.41,472.27,165.80,427.05,1567.53,-181.71314
16,2016,NY,Total Electric Power Industry,Total,1343.76715,1.0,504.90,474.61,166.62,429.17,1575.30,-231.53285
17,2017,NY,Total Electric Power Industry,Total,1280.25386,1.0,493.30,463.71,162.79,419.31,1539.11,-258.85614
18,2018,NY,Total Electric Power Industry,Total,1325.20501,1.0,525.67,494.13,173.47,446.82,1640.09,-314.88499
19,2019,NY,Total Electric Power Industry,Total,1316.03289,1.0,515.36,484.44,170.07,438.06,1607.93,-291.89711
20,2020,NY,Total Electric Power Industry,Total,1294.30271,1.0,524.06,492.62,172.94,445.45,1635.07,-340.76729
21,2021,NY,Total Electric Power Industry,Total,1249.44935,1.0,528.34,496.64,174.35,449.09,1648.42,-398.97065


In [14]:
NYS_Annual_Energy_Generation_and_Sales.to_csv('Resources/NYS_Annual_Energy_Generation_and_Sales.csv')